<a href="https://colab.research.google.com/github/jbpost2/ST-554-Big-Data-with-Python/blob/main/02_Big_Data_Management/03_Databases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Storage and Basic SQL
> Justin Post

---

Before we get into Big Data, we need to understand the most common way to store many different, related, datasets. We've already looked at basic ways to store data in a single file:

- `.csv` (comma separated value)
- `.txt` (generic, usually delimited, data)
- `.dat` (generic, usually delimited, data)
- `.JSON` (think dictionary)

When we have multiple data sets (or sources) a **database** is often used along with a database management system (DBMS).

Consider the diagram below. This shows many different **tables** (think data frames) such as `playlists`, `playlist_track`, and `tracks`. These tables are linked through **keys** such as the `Playlistid` or `Trackid`.

![](https://www4.stat.ncsu.edu/online/datasets/chinook-database-diagram.png)

- The data, the DBMS, and the applications associated with them are often simply called a 'database'
- As the data tables are generally linked, we call this a **relational database**
- The DBMS software is used to create, edit, and maintain database files and records, enabling easier file and record creation, data entry, data editing, updating, and reporting.

In this notebook we explore databases and the basics of the common structured query language (SQL) used to interact with them.

Note: These types of webpages are built from Jupyter notebooks (`.ipynb` files). You can access your own versions of them by [clicking here](https://colab.research.google.com/github/jbpost2/ST-554-Big-Data-with-Python/blob/main/01_Programming_in_python/06-Numeric_Types_Booleans.ipynb). **It is highly recommended that you go through and run the notebooks yourself, modifying and rerunning things where you'd like!**

## Common Database Software

Many common types of relational databases management systems (RDBMS) exist. Some are free and some are not. A few common ones are:

- [Oracle](https://docs.oracle.com/cd/E11882_01/server.112/e40540/intro.htm#CNCPT001)
- [MySQL](https://dev.mysql.com/doc/)
- [SQL Server](https://docs.microsoft.com/en-us/sql/relational-databases/databases/databases?view=sql-server-ver15)
- [PostgreSQL](https://www.postgresql.org/about/)
- [SQLite](https://www.sqlite.org/index.html)
- [Azure SQL](https://docs.microsoft.com/en-us/azure/azure-sql/)

Most RDBMS have their own **Structured Query Language** (SQL), however the basic functionality is similar across them!

## Actions on Databases

There are a few common actions we often want to perform on a database. The acronym CRUD is used to describe four actions:

- Create data
- Read data
- Update data
- Delete data

We will write SQL code to do these actions!

You could imagine that, with multiple users possibly accessing a database, we need to be very careful in how we do these actions. There are [four properties that relational database transactions must have](https://www.oracle.com/database/what-is-a-relational-database/):

- **Atomicity** defines all the elements that make up a complete database transaction.
- **Consistency** defines the rules for maintaining data points in a correct state after a transaction.
- **Isolation** keeps the effect of a transaction invisible to others until it is committed, to avoid confusion.
- **Durability** ensures that data changes become permanent once the transaction is committed.

This is commonly referred to as ACID properties.

Let's explore using `SQLite` with `python`!


## Accessing an SQLite Database in `python`

The `sqlite3` module provides the ability to connect and perform actions on an `SQLite` database.

- We need to import this module
- Then we just need a path to our database file (if we already have one, otherwise one will be created at the path - if possible)
- We'll look at the commonly used `chinook` database the diagram above described.
- As we need to have the ability to write to the database from our notebook, we can't share the same database. That means you'll need to [download the `chinook.db` file](https://www4.stat.ncsu.edu/online/datasets/chinook.db) and upload it to your folder area on the left!
- If you want the changes to remain after you are done, you'd need to download that file before closing the session!


In [1]:
#bring in the module
import sqlite3
#make the connection to the .db file. My file is in the main folder on the left
con = sqlite3.connect("chinook.db")

### Accessing the Table Schema

Every SQLite database contains a "schema table" with information about that database.  This describes the
- tables
- indices (special lookup tables to improve efficiency of queries)
- triggers (named database object that is executed automatically when an INSERT, UPDATE or DELETE statement is issued against the associated table)
- views (read-only tables, combinations of tables, etc.)

A schema file contains one row for each table, index, view, and trigger in the schema.

We can get the schema by issuing an SQL command!  A common SQL command for querying a database looks like this:
- `SELECT column1, column2 FROM table WHERE logical_of_some_kind;`

For `SQLite` from `python`, usually we'll follow this structure:

1. Create a `cursor` object using `cursor = con.cursor()`
2. Write our SQL command as a string
3. Execute the SQL code using the `.execute()` method (`cursor.execute()`) on our `cursor` object
4. Use the `.fetchall()` method (`cursor.fetchall()`) to actually return the data requested
5. Close the conneciton made by the cursor (`cursor.close()`)

- Below we use the multiline comment (three quotation marks to start and end) in order to write more legible SQL code

In [27]:
#create a 'cursor' object from our connection
cursor = con.cursor()

#SQL query to return all table names in the data base
#The * indicates we want to select everything
get_schema = '''
        SELECT *
        FROM sqlite_schema
        WHERE type = "table";
        '''

#execute the SQL query on the database!
cursor.execute(get_schema)

#The information for the query is stored in memory. We use the fetchall() method to actually return the information
result = cursor.fetchall()

#finall we close the connection the cursor made with that query
cursor.close()

We can now look at what is stored in `result`. Here it is actually a `list` that we can cycle through and print information from.

In [28]:
print(type(result))
for i in result:
  print(i)

<class 'list'>
('table', 'albums', 'albums', 2, 'CREATE TABLE "albums"\r\n(\r\n    [AlbumId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [Title] NVARCHAR(160)  NOT NULL,\r\n    [ArtistId] INTEGER  NOT NULL,\r\n    FOREIGN KEY ([ArtistId]) REFERENCES "artists" ([ArtistId]) \r\n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\r\n)')
('table', 'sqlite_sequence', 'sqlite_sequence', 3, 'CREATE TABLE sqlite_sequence(name,seq)')
('table', 'artists', 'artists', 4, 'CREATE TABLE "artists"\r\n(\r\n    [ArtistId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [Name] NVARCHAR(120)\r\n)')
('table', 'customers', 'customers', 5, 'CREATE TABLE "customers"\r\n(\r\n    [CustomerId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\r\n    [FirstName] NVARCHAR(40)  NOT NULL,\r\n    [LastName] NVARCHAR(20)  NOT NULL,\r\n    [Company] NVARCHAR(80),\r\n    [Address] NVARCHAR(70),\r\n    [City] NVARCHAR(40),\r\n    [State] NVARCHAR(40),\r\n    [Country] NVARCHAR(40),\r\n    [PostalCode] NVARCHAR(10),\r\n    [

Often we want to put this information into our common `pandas` data frame format!

In [14]:
import pandas as pd

In [29]:
#create a data frame
schema_df = pd.DataFrame(result)
schema_df

,0,1,2,3,4
0,table,albums,albums,2,"CREATE TABLE ""albums""\r\n(\r\n [AlbumId] IN..."
1,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
2,table,artists,artists,4,"CREATE TABLE ""artists""\r\n(\r\n [ArtistId] ..."
3,table,customers,customers,5,"CREATE TABLE ""customers""\r\n(\r\n [Customer..."
4,table,employees,employees,8,"CREATE TABLE ""employees""\r\n(\r\n [Employee..."
5,table,genres,genres,10,"CREATE TABLE ""genres""\r\n(\r\n [GenreId] IN..."
6,table,invoices,invoices,11,"CREATE TABLE ""invoices""\r\n(\r\n [InvoiceId..."
7,table,invoice_items,invoice_items,13,"CREATE TABLE ""invoice_items""\r\n(\r\n [Invo..."
8,table,media_types,media_types,15,"CREATE TABLE ""media_types""\r\n(\r\n [MediaT..."
9,table,playlists,playlists,16,"CREATE TABLE ""playlists""\r\n(\r\n [Playlist..."


Pretty good, just missing column names. For the [schema table, those are standardized](https://www.sqlite.org/schematab.html).

In [30]:
#current column names
schema_df.columns

RangeIndex(start=0, stop=5, step=1)

In [31]:
schema_df.rename(columns={0: 'type', 1: 'name', 2: 'tbl_name', 3: 'rootpage', 4: 'sql'}, inplace = True)
schema_df

,type,name,tbl_name,rootpage,sql
0,table,albums,albums,2,"CREATE TABLE ""albums""\r\n(\r\n [AlbumId] IN..."
1,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
2,table,artists,artists,4,"CREATE TABLE ""artists""\r\n(\r\n [ArtistId] ..."
3,table,customers,customers,5,"CREATE TABLE ""customers""\r\n(\r\n [Customer..."
4,table,employees,employees,8,"CREATE TABLE ""employees""\r\n(\r\n [Employee..."
5,table,genres,genres,10,"CREATE TABLE ""genres""\r\n(\r\n [GenreId] IN..."
6,table,invoices,invoices,11,"CREATE TABLE ""invoices""\r\n(\r\n [InvoiceId..."
7,table,invoice_items,invoice_items,13,"CREATE TABLE ""invoice_items""\r\n(\r\n [Invo..."
8,table,media_types,media_types,15,"CREATE TABLE ""media_types""\r\n(\r\n [MediaT..."
9,table,playlists,playlists,16,"CREATE TABLE ""playlists""\r\n(\r\n [Playlist..."


Alternatively, we can use the function `read_sql()` from `pandas` to automatically put the result into a data frame!

- Here we don't need a `cursor` object, we just pass the `SQL` string and the connection object.

In [32]:
schema_df2 = pd.read_sql(get_schema, con)
schema_df2

,type,name,tbl_name,rootpage,sql
0,table,albums,albums,2,"CREATE TABLE ""albums""\r\n(\r\n [AlbumId] IN..."
1,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
2,table,artists,artists,4,"CREATE TABLE ""artists""\r\n(\r\n [ArtistId] ..."
3,table,customers,customers,5,"CREATE TABLE ""customers""\r\n(\r\n [Customer..."
4,table,employees,employees,8,"CREATE TABLE ""employees""\r\n(\r\n [Employee..."
5,table,genres,genres,10,"CREATE TABLE ""genres""\r\n(\r\n [GenreId] IN..."
6,table,invoices,invoices,11,"CREATE TABLE ""invoices""\r\n(\r\n [InvoiceId..."
7,table,invoice_items,invoice_items,13,"CREATE TABLE ""invoice_items""\r\n(\r\n [Invo..."
8,table,media_types,media_types,15,"CREATE TABLE ""media_types""\r\n(\r\n [MediaT..."
9,table,playlists,playlists,16,"CREATE TABLE ""playlists""\r\n(\r\n [Playlist..."


Ahh, that's better!

### Querying a Table

Now that we know which tables exist, we can query them! Let's return all the albums in the `albums` table.

We go through the same process as before (as we closed our `cursor` object we need to create a new one):

1. Create our cursor using `con.cursor()`
2. Write our SQL command as a string
3. Execute the SQL code using the `.execute()` method (`cursor.execute()`) on our `cursor` object
4. Use the `.fetchall()` method (`cursor.fetchall()`) to actually return the data requested
5. Close the conneciton made by the cursor (`cursor.close()`)

The `FROM` function allows us to choose which table to query. `LIMIT 20` says to limit what is returned to 20 rows.

In [33]:
#create the cursor instance
cursor = con.cursor()
#create the SQL string
get_albums = '''
        SELECT *
        FROM albums
        LIMIT 20;
        '''
#execute the query
albums = cursor.execute(get_albums)
#grab the results
album_results = albums.fetchall()
#close the cursor
cursor.close()
#check the results
album_results

[(1, 'For Those About To Rock We Salute You', 1),
 (2, 'Balls to the Wall', 2),
 (3, 'Restless and Wild', 2),
 (4, 'Let There Be Rock', 1),
 (5, 'Big Ones', 3),
 (6, 'Jagged Little Pill', 4),
 (7, 'Facelift', 5),
 (8, 'Warner 25 Anos', 6),
 (9, 'Plays Metallica By Four Cellos', 7),
 (10, 'Audioslave', 8),
 (11, 'Out Of Exile', 8),
 (12, 'BackBeat Soundtrack', 9),
 (13, 'The Best Of Billy Cobham', 10),
 (14, 'Alcohol Fueled Brewtality Live! [Disc 1]', 11),
 (15, 'Alcohol Fueled Brewtality Live! [Disc 2]', 11),
 (16, 'Black Sabbath', 12),
 (17, 'Black Sabbath Vol. 4 (Remaster)', 12),
 (18, 'Body Count', 13),
 (19, 'Chemical Wedding', 14),
 (20, 'The Best Of Buddy Guy - The Millenium Collection', 15)]

Ok, when just reading a table, let's go with `pd.read_sql()` instead since it returns things in a nicer format!

In [34]:
album_results = pd.read_sql(get_albums, con)
album_results

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
5,6,Jagged Little Pill,4
6,7,Facelift,5
7,8,Warner 25 Anos,6
8,9,Plays Metallica By Four Cellos,7
9,10,Audioslave,8


## Process for doing a CRUD activity

(**C**reate, **R**ead, **U**pdate, **D**elete)

If we wanted to do something other than just read a table, we need to follow our general structure from above:

- Create a connection using `sqlite3.connect(path)`
- Create a `cursor` [object](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor) associated with the connection
- Write an SQL query as a string
- Use `cursor.execute()` (or `cursor.executemany()`) to execute the SQL
- Close the `cursor` object

## Common SQL commands

- CREATE TABLE - creates a new table
- INSERT INTO - adds records to a table
- UPDATE - modify existing records
- DELETE FROM - deletes data
- DROP TABLE - removes a table
- SELECT - reads data (use `fetchone()`, `fetchall()`, or the returned value as an iterator but we'll use `pd.read_sql()` for simple read commands)

Also many important **joins** we'll cover shortly.

Let's go through a few of these actions to show how we can write our SQL code and execute it! **Note that SQL code is not case sensitive.**

### Create a Table

Let's start by creating a table.  We write the SQL code and specify the name of the table and and variables we want to create.

- We specify the type of data the variable will hold after naming the variable
- Check [this reference](https://www.sqlite.org/datatype3.html) for possible data types!

In [35]:
#create the cursor instance
cursor = con.cursor()
#write our SQL to create a table
#here we also specify two variables: album and artist along with the type of data they'll hold
ct = """
    CREATE TABLE IF NOT EXISTS justin_music (
        album TEXT,
        artist TEXT);
"""
#execute the SQL code
cursor.execute(ct)

Let's check that it worked before we close the connection!

In [37]:
pd.read_sql('''
        SELECT *
        FROM justin_music;
        ''', con)

,album,artist


Cool, now let's close the connection.

In [38]:
cursor = con.cursor()

You can see we'll do this process a lot.  It may be best to create a function to help us out.  (Modified from https://realpython.com/python-sql-libraries/)

In [40]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
    cursor.close()

The function will execute the query for us and close the cursor.

In [43]:
execute_query(con, ct)

Query executed successfully


### INSERT INTO

Now lets add some data to our table using `INSERT INTO`.  We need to pass the table name, optionally the columns we'll specify, and the values to fill with.

In [44]:
create_rows = """
       INSERT INTO
           justin_music (album, artist)
       VALUES
           ("Sixteen Stone", "Bush"),
           ("Listener Supported", "Dave Matthews Band"),
           ("Chris Stapleton", "Traveler"),
           ("1989", "Taylor Swift");
"""
execute_query(con, create_rows)

Query executed successfully


Let's check if it worked!

In [47]:
pd.read_sql("SELECT * FROM justin_music", con)

,album,artist
0,Sixteen Stone,Bush
1,Listener Supported,Dave Matthews Band
2,Chris Stapleton,Traveler
3,1989,Taylor Swift


Now let's add a row with only an artist.

In [48]:
create_row = """
       INSERT INTO
           justin_music (artist)
       VALUES
           ("Taylor Swift");
"""

execute_query(con, create_row)

Query executed successfully


This inserts a `None` into the data frame but note that the value in the original table is actually a `NULL`.

In [51]:
pd.read_sql("SELECT * FROM justin_music", con)

,album,artist
0,Sixteen Stone,Bush
1,Listener Supported,Dave Matthews Band
2,Chris Stapleton,Traveler
3,1989,Taylor Swift
4,None,Taylor Swift


### UPDATE

We can use `update` to modify an existing row of data.  We use

- UPDATE to specify the table
- SET to specify the values of the columns
- WHERE to describe which records (rows) to modify

We can use `AND`, `OR`, `IN`, and other logical operators in our SQL code! (See [Operators and Parse-Affecting Attributes](https://www.sqlite.org/lang_expr.html))

In [52]:
mod_row = """
    UPDATE justin_music
    SET album = "Red (Taylor's Version)", artist = "Taylor Swift"
    WHERE (album is null) AND (artist = "Taylor Swift");
"""

execute_query(con, mod_row)

Query executed successfully


In [53]:
pd.read_sql("SELECT * FROM justin_music", con)

,album,artist
0,Sixteen Stone,Bush
1,Listener Supported,Dave Matthews Band
2,Chris Stapleton,Traveler
3,1989,Taylor Swift
4,Red (Taylor's Version),Taylor Swift


### DELETE FROM

We can also delete certain rows.  We use

- DELETE FROM to specify the table
- a WHERE condition to determine what is deleted


In [54]:
del_row = """
    DELETE FROM justin_music
    WHERE artist = "Taylor Swift";
    """

execute_query(con, del_row)

Query executed successfully


In [55]:
pd.read_sql("SELECT * FROM justin_music", con)

,album,artist
0,Sixteen Stone,Bush
1,Listener Supported,Dave Matthews Band
2,Chris Stapleton,Traveler


### DROP TABLE

And of course we can remove an entire table using DROP TABLE.  Just specify the table to remove!

In [56]:
execute_query(con, "DROP TABLE justin_music")

Query executed successfully


In [57]:
pd.read_sql(get_schema, con)

,type,name,tbl_name,rootpage,sql
0,table,albums,albums,2,"CREATE TABLE ""albums""\r\n(\r\n [AlbumId] IN..."
1,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
2,table,artists,artists,4,"CREATE TABLE ""artists""\r\n(\r\n [ArtistId] ..."
3,table,customers,customers,5,"CREATE TABLE ""customers""\r\n(\r\n [Customer..."
4,table,employees,employees,8,"CREATE TABLE ""employees""\r\n(\r\n [Employee..."
5,table,genres,genres,10,"CREATE TABLE ""genres""\r\n(\r\n [GenreId] IN..."
6,table,invoices,invoices,11,"CREATE TABLE ""invoices""\r\n(\r\n [InvoiceId..."
7,table,invoice_items,invoice_items,13,"CREATE TABLE ""invoice_items""\r\n(\r\n [Invo..."
8,table,media_types,media_types,15,"CREATE TABLE ""media_types""\r\n(\r\n [MediaT..."
9,table,playlists,playlists,16,"CREATE TABLE ""playlists""\r\n(\r\n [Playlist..."


My table is gone :(

### SELECT

As a statistician, you'd spend most of your time pulling data of interest and then summarizing it, running models, etc.  

`SELECT` is the workhorse for that task! As we are just reading in data from the database, we can just use `pd.read_sql()`.

With SELECT we

- SELECT the column(s) we want
- FROM the table of interest
- Specifing the records (rows) of interest with a WHERE

First, let's see a way to look at all columns in a table.

In [59]:
pd.read_sql('SELECT * FROM albums', con)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
...,...,...,...
342,343,Respighi:Pines of Rome,226
343,344,Schubert: The Late String Quartets & String Qu...,272
344,345,Monteverdi: L'Orfeo,273
345,346,Mozart: Chamber Music,274


Now let's select the Title and ArtistID columns only.  `LIMIT` is a good way to only return (up to) a certain number of results.

In [60]:
pd.read_sql("SELECT Title, artistID FROM albums LIMIT 10;", con)

,Title,ArtistId
0,For Those About To Rock We Salute You,1
1,Balls to the Wall,2
2,Restless and Wild,2
3,Let There Be Rock,1
4,Big Ones,3
5,Jagged Little Pill,4
6,Facelift,5
7,Warner 25 Anos,6
8,Plays Metallica By Four Cellos,7
9,Audioslave,8


WHERE allows us to specify conditions on rows.

In [ ]:
pd.read_sql"SELECT * FROM albums WHERE artistID = 2 LIMIT 10;").fetchall()

[(2, 'Balls to the Wall', 2), (3, 'Restless and Wild', 2)]

Ok, so the above is straight SQL code.  That's a great skill to learn.  We can easily switch to using `pandas` to turn the result into easy to work with data frames!

In [ ]:
import pandas as pd

The function `pd.read_sql(sql, con)` returns the results as a data frame.

In [ ]:
pd.read_sql("SELECT Title, artistID FROM albums LIMIT 10;", con)

,Title,ArtistId
0,For Those About To Rock We Salute You,1
1,Balls to the Wall,2
2,Restless and Wild,2
3,Let There Be Rock,1
4,Big Ones,3
5,Jagged Little Pill,4
6,Facelift,5
7,Warner 25 Anos,6
8,Plays Metallica By Four Cellos,7
9,Audioslave,8


In [ ]:
pd.read_sql("SELECT * FROM tracks", con)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99


Niiiice! That's now going to be very easy to summarize and plot.

## Clean Up

If we want to save any changes made to our database, we need to do a `commit()` to the connection via `con.commit()`.  This saves the changes made and releases any locks on the data.  Other connections to the database can only see changes you've made if you commit them.

When you are doing working you should also close your connection.  This is done with the `close()` method.

In [ ]:
con.close()

We skimmed over some of the syntax that SQL follows.  You should read over the [arithmetic operators](https://www.w3resource.com/sqlite/arithmetic-operators.php), [comparison operators](https://www.w3resource.com/sqlite/comparison-operators.php), and the other operators linked on the left of that page.

# SQL Joins on chinook

Now let's do some joins on the chinook database.  We'll reread in the packages are make our connection to the database.

In [ ]:
import sqlite3
import pandas as pd
con = sqlite3.connect("chinook.db")

![](https://drive.google.com/uc?export=view&id=1uk0kQci8_fJdXoBJ4SuEx9KeG9eEP6Gf)

Using pandas we can check out the tables returned as a data frame.

In [ ]:
pd.read_sql("SELECT * FROM sqlite_schema WHERE type = 'table';", con)

,type,name,tbl_name,rootpage,sql
0,table,albums,albums,2,"CREATE TABLE ""albums""\r\n(\r\n [AlbumId] IN..."
1,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
2,table,artists,artists,4,"CREATE TABLE ""artists""\r\n(\r\n [ArtistId] ..."
3,table,customers,customers,5,"CREATE TABLE ""customers""\r\n(\r\n [Customer..."
4,table,employees,employees,8,"CREATE TABLE ""employees""\r\n(\r\n [Employee..."
5,table,genres,genres,10,"CREATE TABLE ""genres""\r\n(\r\n [GenreId] IN..."
6,table,invoices,invoices,11,"CREATE TABLE ""invoices""\r\n(\r\n [InvoiceId..."
7,table,invoice_items,invoice_items,13,"CREATE TABLE ""invoice_items""\r\n(\r\n [Invo..."
8,table,media_types,media_types,15,"CREATE TABLE ""media_types""\r\n(\r\n [MediaT..."
9,table,playlists,playlists,16,"CREATE TABLE ""playlists""\r\n(\r\n [Playlist..."


The tracks and albums tables each share a variable/column/key of `AlbumId`.  Let's do an inner join on those using `pd.merge()`.  Remember that we can pull the full tables into pandas using `pd.read_sql()` and then use `pandas` to do the join!

In [ ]:
tracks_albums = pd.merge(left = pd.read_sql("SELECT * FROM tracks", con), right = pd.read_sql("SELECT * FROM albums", con),
         how = "inner",
         on = "AlbumId")
tracks_albums

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,Title,ArtistId
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,For Those About To Rock We Salute You,1
1,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99,For Those About To Rock We Salute You,1
2,7,Let's Get It Up,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99,For Those About To Rock We Salute You,1
3,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99,For Those About To Rock We Salute You,1
4,9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99,For Those About To Rock We Salute You,1
...,...,...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99,Respighi:Pines of Rome,226
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99,Schubert: The Late String Quartets & String Qu...,272
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99,Monteverdi: L'Orfeo,273
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99,Mozart: Chamber Music,274


Now we'll join that table we just made with another table, `invoice_items` and do a left join (treating `invoice_items` as the left table).  We'll join on the key that is shared, `trackId`.

In [ ]:
next = pd.merge(left = pd.read_sql("SELECT * FROM invoice_items", con),
         right= tracks_albums,
         how = "left",
         on = "TrackId")
next

,InvoiceLineId,InvoiceId,TrackId,UnitPrice_x,Quantity,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice_y,Title,ArtistId
0,1,1,2,0.99,1,Balls to the Wall,2,2,1,None,342562,5510424,0.99,Balls to the Wall,2
1,2,1,4,0.99,1,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99,Restless and Wild,2
2,3,2,6,0.99,1,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99,For Those About To Rock We Salute You,1
3,4,2,8,0.99,1,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99,For Those About To Rock We Salute You,1
4,5,2,10,0.99,1,Evil Walks,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",263497,8611245,0.99,For Those About To Rock We Salute You,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,2236,411,3136,0.99,1,Looking For Love,141,1,3,Sykes,391941,12769847,0.99,Greatest Hits,100
2236,2237,411,3145,0.99,1,Sweet Lady Luck,141,1,3,Vandenberg,273737,8919163,0.99,Greatest Hits,100
2237,2238,411,3154,0.99,1,Feirinha da Pavuna/Luz do Repente/Bagaço da La...,248,1,7,"Arlindo Cruz/Franco/Marquinhos PQD/Negro, Jove...",107206,3593684,0.99,Ao Vivo [IMPORT],155
2238,2239,411,3163,0.99,1,Samba pras moças,248,1,7,Grazielle/Roque Ferreira,152816,5121366,0.99,Ao Vivo [IMPORT],155


Lastly, we'll now combine that table with the `invoices` table using an outer join on the key `InvoiceId`.  Note that you can actually do all of these joins in SQL using one call (which is likely much more efficient!).  Please see the additional readings for topic 2 for more information.

In [ ]:
pd.merge(left = pd.read_sql("SELECT * FROM invoices", con),
         right = next,
         how = "outer",
         on = "InvoiceId")

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total,InvoiceLineId,...,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice_y,Title,ArtistId
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,1,...,Balls to the Wall,2,2,1,None,342562,5510424,0.99,Balls to the Wall,2
1,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,2,...,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99,Restless and Wild,2
2,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,3,...,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99,For Those About To Rock We Salute You,1
3,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,4,...,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99,For Those About To Rock We Salute You,1
4,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,5,...,Evil Walks,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",263497,8611245,0.99,For Those About To Rock We Salute You,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,411,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86,2236,...,Looking For Love,141,1,3,Sykes,391941,12769847,0.99,Greatest Hits,100
2236,411,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86,2237,...,Sweet Lady Luck,141,1,3,Vandenberg,273737,8919163,0.99,Greatest Hits,100
2237,411,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86,2238,...,Feirinha da Pavuna/Luz do Repente/Bagaço da La...,248,1,7,"Arlindo Cruz/Franco/Marquinhos PQD/Negro, Jove...",107206,3593684,0.99,Ao Vivo [IMPORT],155
2238,411,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86,2239,...,Samba pras moças,248,1,7,Grazielle/Roque Ferreira,152816,5121366,0.99,Ao Vivo [IMPORT],155
